# Projeto MegaDados

**Alunos:**
    
+ Rafael Almada
+ Warlen Cesar Rodrigues
    
**Professor:** Fábio Ayres

In [1]:
import pyspark
from math import log10

In [2]:
sc = pyspark.SparkContext(appName='Projeto e-sports')

In [3]:
sc

<SparkContext master=local[*] appName=Projeto e-sports>

In [4]:
rdd = sc.sequenceFile('part-00000')
rdd

MapPartitionsRDD[2] at mapPartitions at SerDeUtil.scala:244

In [5]:
# result = rdd.take(10)
# result[0]

### Montando RDD para Valorant

In [6]:
#Pegar os documentos que possuem a palavra Valorant
def getValorantWord(item):
    if 'Valorant' in item[1]:
        return item
    
rdd_valorant = rdd.filter(getValorantWord)
rdd_valorant.count()

10

### Montando RDD para Fortnite

In [7]:
#Pegar os documentos que possuem a palavra Counter Strike
def getFortnitetWord(item):
    if 'Fortnite' in item[1]:
        return item
    
rdd_fortnite = rdd.filter(getFortnitetWord)
rdd_fortnite.count()

61

### Operacionalização para Intersecção entre Valorant e Fortnite

In [8]:
#Pegar somente os documentos que possuem as duas palavras 
rdd_valorant_fortnite = rdd_valorant.intersection(rdd_fortnite)
rdd_valorant_fortnite.count()

3

#### Operacionalizando frequência de palavras (1 doc = 1 contagem)

In [9]:
def conta_palavras(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual

rdd_valorant_fortnite_doc_freq = rdd_valorant_fortnite.flatMap(conta_palavras).reduceByKey(junta_contagens)

In [10]:
#Listando tuplas (palavra, qtde. de Docs) para intersecção de Valorant e CSGO
#Neste cenário, o valor máximo do value é a quantidade retornada por rdd_valorant_csgo.count()
result = rdd_valorant_fortnite_doc_freq.collect() 
sorted(result, key=lambda x : -x[1])

[('Legends', 3),
 ('of', 3),
 ('Fortnite', 3),
 ('o', 3),
 ('Riot', 3),
 ('para', 3),
 ('na', 3),
 ('sobre', 3),
 ('a', 3),
 ('-', 3),
 ('League', 3),
 ('e', 3),
 ('da', 3),
 ('Games', 3),
 ('do', 3),
 ('com', 3),
 ('as', 3),
 ('skins', 3),
 ('Valorant', 3),
 ('de', 3),
 ('no', 2),
 ('Como', 2),
 ('Fire', 2),
 ('Jogos', 2),
 ('Todos', 2),
 ('CBLOL', 2),
 ('Online', 2),
 ('é', 2),
 ('em', 2),
 ('vence', 2),
 ('semifinais', 2),
 ('será', 2),
 ('não', 2),
 ('por', 2),
 ('anuncia', 2),
 ('semanas', 2),
 ('sua', 2),
 ('novos', 2),
 ('–', 2),
 ('os', 2),
 ('que', 2),
 ('chega', 2),
 ('até', 2),
 ('360', 2),
 ('Worlds', 2),
 ('Wild', 2),
 ('como', 2),
 ('Free', 2),
 ('seu', 2),
 ('GTA', 2),
 ('Impact', 2),
 ('Confira', 2),
 ('à', 2),
 ('©', 2),
 ('The', 2),
 ('LoL:', 2),
 ('mais', 2),
 ('Seraphine', 2),
 ('atrás', 2),
 ('15', 2),
 ('estão', 2),
 ('está', 2),
 ('R$', 2),
 ('apenas', 2),
 ('Mobile', 2),
 ('CS:GO', 2),
 ('online', 2),
 ('2020', 2),
 ('confirmado', 2),
 ('2ª', 2),
 ('Consoles', 2

#### Fazendo o recorte de palavras muito específicas ou muito comuns

In [11]:
N = rdd.count()
DOC_COUNT_MIN = 2 # Isso deve mudar de amostra para amostra
DOC_COUNT_MAX = 0.7 * N # Isso deve mudar de amostra para amostra (0.7 * rdd.count())

def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < DOC_COUNT_MAX) & (contagem >= DOC_COUNT_MIN)

rdd_valorant_fortnite_freq_filtrado = rdd_valorant_fortnite_doc_freq.filter(filtra_doc_freq)

In [12]:
rdd_valorant_fortnite_freq_filtrado.take(10)

[('no', 2),
 ('Como', 2),
 ('Legends', 3),
 ('of', 3),
 ('Fortnite', 3),
 ('Fire', 2),
 ('Jogos', 2),
 ('Todos', 2),
 ('CBLOL', 2),
 ('o', 3)]

#### Calculo do IDF para as palavras encontradas em documentos que possuem Valorant E Fortnite

In [13]:
def idf(item):
    palavra, contagem = item
    return (palavra, log10(N/contagem))

rdd_idf = rdd_valorant_fortnite_freq_filtrado.map(idf)
result = rdd_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('Legends', 4.0807827663379035),
 ('of', 4.0807827663379035),
 ('Fortnite', 4.0807827663379035),
 ('o', 4.0807827663379035),
 ('Riot', 4.0807827663379035),
 ('para', 4.0807827663379035),
 ('na', 4.0807827663379035),
 ('sobre', 4.0807827663379035),
 ('a', 4.0807827663379035),
 ('-', 4.0807827663379035),
 ('League', 4.0807827663379035),
 ('e', 4.0807827663379035),
 ('da', 4.0807827663379035),
 ('Games', 4.0807827663379035),
 ('do', 4.0807827663379035),
 ('com', 4.0807827663379035),
 ('as', 4.0807827663379035),
 ('skins', 4.0807827663379035),
 ('Valorant', 4.0807827663379035),
 ('de', 4.0807827663379035),
 ('no', 4.256874025393585),
 ('Como', 4.256874025393585),
 ('Fire', 4.256874025393585),
 ('Jogos', 4.256874025393585),
 ('Todos', 4.256874025393585),
 ('CBLOL', 4.256874025393585),
 ('Online', 4.256874025393585),
 ('é', 4.256874025393585),
 ('em', 4.256874025393585),
 ('vence', 4.256874025393585),
 ('semifinais', 4.256874025393585),
 ('será', 4.256874025393585),
 ('não', 4.2568740253935

#### Calculo da frequencia e relevância das palavras

In [14]:
def freq_palavra(item):
    palavra, contagem = item
    return (palavra, log10(1+contagem))

def relevancia(item):
    palavra, freq = item
    freq_calc = freq_palavra(item)
    idf_calc = idf(item)
    relev = freq_calc[1]*idf_calc[1]
    return (palavra, relev)

In [15]:
freq_filtrado_sort_by_key_rdd = rdd_valorant_fortnite_freq_filtrado.sortByKey()
rdd_relevancia_ambas_palavras_mesmo_doc = freq_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_ambas_palavras_mesmo_doc.collect()
sorted(res, key=lambda x : x[1])[-100:]

[('15', 2.0310450761793275),
 ('2020', 2.0310450761793275),
 ('2020:', 2.0310450761793275),
 ('2ª', 2.0310450761793275),
 ('360', 2.0310450761793275),
 ('CBLOL', 2.0310450761793275),
 ('CS:GO', 2.0310450761793275),
 ('Como', 2.0310450761793275),
 ('Confira', 2.0310450761793275),
 ('Consoles', 2.0310450761793275),
 ('Epic', 2.0310450761793275),
 ('Etapa', 2.0310450761793275),
 ('Fire', 2.0310450761793275),
 ('Free', 2.0310450761793275),
 ('GTA', 2.0310450761793275),
 ('Genshin', 2.0310450761793275),
 ('Impact', 2.0310450761793275),
 ('Jogos', 2.0310450761793275),
 ('LBFF', 2.0310450761793275),
 ('LoL:', 2.0310450761793275),
 ('Mobile', 2.0310450761793275),
 ('Online', 2.0310450761793275),
 ('Outros', 2.0310450761793275),
 ('PUBG', 2.0310450761793275),
 ('Play', 2.0310450761793275),
 ('R$', 2.0310450761793275),
 ('Rift', 2.0310450761793275),
 ('Seraphine', 2.0310450761793275),
 ('The', 2.0310450761793275),
 ('Todos', 2.0310450761793275),
 ('Wild', 2.0310450761793275),
 ('Worlds', 2.03104

### Operacionalização para exclusão mútua entre Valorant e Fortnite
### Fortnite

#### Criando o RDD com documentos que possuem somente Fortnite
#### Filtrando o RDD

In [16]:
rdd_fortnite_only = rdd_fortnite.subtract(rdd_valorant_fortnite)
rdd_fortnite_only_doc_freq = rdd_fortnite_only.flatMap(conta_palavras).reduceByKey(junta_contagens)
rdd_fortnite_only_doc_freq_filtrado = rdd_fortnite_only_doc_freq.filter(filtra_doc_freq)
rdd_fortnite_only_doc_freq_filtrado.take(10)

[('Azul', 19),
 ('FECHAR', 2),
 ('acima', 14),
 ('Fortnite', 52),
 ('Sharingan', 2),
 ('Guardião', 2),
 ('das', 42),
 ('perucas', 2),
 ('Pokemon', 10),
 ('of', 25)]

#### Calculando o IDF das palavras

In [17]:
rdd_fortnite_only_idf = rdd_fortnite_only_doc_freq_filtrado.map(idf)
result = rdd_fortnite_only_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('surto', 4.256874025393585),
 ('Nem', 4.256874025393585),
 ('jogabilidade', 4.256874025393585),
 ('selecionar', 4.256874025393585),
 ('Ray', 4.256874025393585),
 ('Detour', 4.256874025393585),
 ('109,90', 4.256874025393585),
 ('escala', 4.256874025393585),
 ('ocasião,', 4.256874025393585),
 ('profissionais', 4.256874025393585),
 ('tiro', 4.256874025393585),
 ('Continue', 4.256874025393585),
 ('aponta', 4.256874025393585),
 ('área', 4.256874025393585),
 ('Vários', 4.256874025393585),
 ('face', 4.256874025393585),
 ('420', 4.256874025393585),
 ('administração', 4.256874025393585),
 ('tutorial', 4.256874025393585),
 ('legenda', 4.256874025393585),
 ('contou', 4.256874025393585),
 ('through', 4.256874025393585),
 ('idade', 4.256874025393585),
 ('elementos', 4.256874025393585),
 ('cupons', 4.256874025393585),
 ('VeículoPlaysetPelúcias', 4.256874025393585),
 ('Derrapante', 4.256874025393585),
 ('Órgãos', 4.256874025393585),
 ('MitsubishiPilotZebraPentelTombowStaedtler', 4.256874025393585),

#### Calculando a relevância das palavras

In [18]:
fortnite_filtrado_sort_by_key_rdd = rdd_fortnite_only_doc_freq_filtrado.sortByKey()
rdd_relevancia_fortnite_only = fortnite_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_fortnite_only.collect()
sorted(res, key=lambda x : x[1])[-100:]

[('online', 4.306301676893127),
 ('sem', 4.306301676893127),
 ('semana', 4.306301676893127),
 ('sociais', 4.306301676893127),
 ('todos', 4.306301676893127),
 ('às', 4.306301676893127),
 ('2019', 4.343656564606857),
 ('Comprar', 4.343656564606857),
 ('Disney', 4.343656564606857),
 ('Home', 4.343656564606857),
 ('Homem', 4.343656564606857),
 ('Início', 4.343656564606857),
 ('Justiça', 4.343656564606857),
 ('Mundo', 4.343656564606857),
 ('Social', 4.343656564606857),
 ('direitos', 4.343656564606857),
 ('entre', 4.343656564606857),
 ('tem', 4.343656564606857),
 ('As', 4.378610447193646),
 ('Black', 4.378610447193646),
 ('Pagamento', 4.378610447193646),
 ('Star', 4.378610447193646),
 ('Super', 4.378610447193646),
 ('seus', 4.378610447193646),
 ('suas', 4.378610447193646),
 ('/', 4.411398301083086),
 ('4', 4.411398301083086),
 ('Conta', 4.411398301083086),
 ('Todos', 4.411398301083086),
 ('produto', 4.411398301083086),
 ('quando', 4.411398301083086),
 ('ser', 4.411398301083086),
 ('10', 4.44

### Valorant

#### Criando o RDD com documentos que possuem somente Valorant
#### Filtrando o RDD

In [19]:
rdd_valorant_only = rdd_valorant.subtract(rdd_valorant_fortnite)
rdd_valorant_only_doc_freq = rdd_valorant_only.flatMap(conta_palavras).reduceByKey(junta_contagens)
rdd_valorant_only_doc_freq_filtrado = rdd_valorant_only_doc_freq.filter(filtra_doc_freq)
rdd_valorant_only_doc_freq_filtrado.take(10)

[('boa', 2),
 ('baseado', 2),
 ('das', 3),
 ('no', 6),
 ('favorito', 2),
 ('podem', 3),
 ('of', 4),
 ('melhor', 3),
 ('mesmo', 3),
 ('listou', 2)]

#### Calculando o IDF das palavras

In [20]:
rdd_valorant_only_idf = rdd_valorant_only_doc_freq_filtrado.map(idf)
result = rdd_valorant_only_idf.collect()
sorted(result, key=lambda x : x[1])[-100:]

[('Mouse', 4.256874025393585),
 ('Compra', 4.256874025393585),
 ('exclusiva', 4.256874025393585),
 ('Centers', 4.256874025393585),
 ('Seja', 4.256874025393585),
 ('Resident', 4.256874025393585),
 ('Avakin', 4.256874025393585),
 ('relativamente', 4.256874025393585),
 ('Colunistas', 4.256874025393585),
 ('esta', 4.256874025393585),
 ('one', 4.256874025393585),
 ('viole', 4.256874025393585),
 ('Thiago', 4.256874025393585),
 ('preços', 4.256874025393585),
 ('gosta', 4.256874025393585),
 ('eles', 4.256874025393585),
 ('séries', 4.256874025393585),
 ('Desfazer', 4.256874025393585),
 ('R$0,00', 4.256874025393585),
 ('Item', 4.256874025393585),
 ('Talvez', 4.256874025393585),
 ('Ragnarok', 4.256874025393585),
 ('Outros', 4.256874025393585),
 ('Star', 4.256874025393585),
 ('Home', 4.256874025393585),
 ('adiado', 4.256874025393585),
 ('novo', 4.256874025393585),
 ('globo.com', 4.256874025393585),
 ('g7', 4.256874025393585),
 ('site.', 4.256874025393585),
 ('Claro,', 4.256874025393585),
 ('meses'

#### Calculando a relevância das palavras

In [21]:
valorant_filtrado_sort_by_key_rdd = rdd_valorant_only_doc_freq_filtrado.sortByKey()
rdd_relevancia_valorant_only = valorant_filtrado_sort_by_key_rdd.map(relevancia)
res = rdd_relevancia_valorant_only.collect()
sorted(res, key=lambda x : x[1])[-100:]

[('tão', 2.4568760369126967),
 ('vez', 2.4568760369126967),
 ('vídeos', 2.4568760369126967),
 ('época', 2.4568760369126967),
 ('Artigos', 2.7650163186127155),
 ('Como', 2.7650163186127155),
 ('Dicas', 2.7650163186127155),
 ('Imagens', 2.7650163186127155),
 ('Já', 2.7650163186127155),
 ('Mobile', 2.7650163186127155),
 ('Nintendo', 2.7650163186127155),
 ('O', 2.7650163186127155),
 ('Para', 2.7650163186127155),
 ('Política', 2.7650163186127155),
 ('Simuladores', 2.7650163186127155),
 ('TV', 2.7650163186127155),
 ('Tutoriais', 2.7650163186127155),
 ('Um', 2.7650163186127155),
 ('WhatsApp', 2.7650163186127155),
 ('World', 2.7650163186127155),
 ('algumas', 2.7650163186127155),
 ('ao', 2.7650163186127155),
 ('até', 2.7650163186127155),
 ('cada', 2.7650163186127155),
 ('casa', 2.7650163186127155),
 ('conta', 2.7650163186127155),
 ('disponíveis', 2.7650163186127155),
 ('história', 2.7650163186127155),
 ('ideia', 2.7650163186127155),
 ('jogos', 2.7650163186127155),
 ('já', 2.7650163186127155),
 

In [22]:
sc.stop()